# DicoCo

Maxime Charruyer

In [82]:
# import necessaire à la création des widgets
import os
import re
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown, Audio
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np


In [83]:
cheminDuFichierCSV = "./../data/dataframe/dataframe.csv" 


In [84]:
buchanan = pd.read_csv(cheminDuFichierCSV, delimiter=",", encoding="utf8")


In [85]:
buchanan

,Unnamed: 0,hwd,pronSampa,ipa,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette
0,0,abacus,{ b @ k @ s,æbəkəs,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus
1,1,abandon,@ b { n d @ n,əbændən,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon
2,2,abandoned,@ b { n d @ n d,əbændənd,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned
3,3,abash,@ b { S,əbæʃ,2,ăbăʹſh,F.,NaN,NaN,v.,9.0,19,abash
4,4,abate,@ b eI t,əb?t,2,ăbāʹte,F.,NaN,NaN,v.,10.0,19,abate
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10328,10328,zenith,z i: n @ T,ziːnəθ,2,zĕʹnith,NaN,NaN,NaN,n.,11.0,241,zenith
10329,10329,zest,z E s t,zɛst,1,zĕſt,NaN,NaN,NaN,n.,13.0,241,zest
10330,10330,zink,z I N k,zɪŋk,1,zĭnk,NaN,NaN,NaN,n.,14.0,241,zink
10331,10331,zodiac,z @U 4 i: { k,zoʊɾiːæk,3,zōʹdĭac,NaN,NaN,NaN,n.,16.0,241,zodiac


In [86]:
session = HTMLSession()
#response = session.get("https://www.ldoceonline.com/dictionary/cochlea")

In [87]:
def getAudioInfo(mot):
    response = session.get("https://dictionary.cambridge.org/fr/dictionnaire/anglais/"+mot)
    html_doc = response.content
    soup = BeautifulSoup(html_doc, 'html.parser')
    words = soup.findAll("div", {"class": "pos-header dpos-h"})
    
    words_list = []
    
    for word in words:
        
        row = []

        #Rédupération de toutes les informations
        audioAll = word.findAll(type="audio/mpeg")
        posAll = word.findAll('span', {'class': 'pos dpos'})
        ipaAll = word.findAll('span', {'class': 'ipa dipa lpr-2 lpl-1'})
        
        #Récupération de pos et ipa
        row.append(posAll[0].text)
        row.append(ipaAll[0].text)
        
        #Récupération du chemin mp3 selon la version uk/us
        for a in audioAll:
            if "uk_pron" in a.get_attribute_list('src')[0]:
                row.append("https://dictionary.cambridge.org/"+a.get_attribute_list('src')[0])
            elif len(audioAll)==1:
                row.append("")
            
            if "us_pron" in a.get_attribute_list('src')[0]:
                row.append("https://dictionary.cambridge.org/"+a.get_attribute_list('src')[0])
            elif len(audioAll)==1:
                row.append("")
                
                
        words_list.append(row)
    
    df = pd.DataFrame.from_records(words_list, columns=["pos", "ipa", "audio_uk", "audio_us"])
    
    return df

In [88]:
#Definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='Entrer un mot',
       layout={'width': '200px'} )

#Definition du widget permettant de taper un texte pour l'ipa
ipaTarget = widgets.Text(
       value="",
       description='IPA',
       layout={'width': '200px'} )

#Sélection d'une plage de valeurs
rangeSyllMin = 1
rangeSyllMax = 27
optionsNbSyll = range(1, 27)
wid_nbsyll = widgets.SelectionRangeSlider(
    options=optionsNbSyll,
    index=(min(optionsNbSyll)-1,max(optionsNbSyll)-1),
    description='Nb de syl',
    disabled=False
)

nbResultMax = 20
wid_nbResult = widgets.IntSlider(
    value=nbResultMax,
    min=1,
    max=nbResultMax,
    step=1,
    description='Nb résultats max',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [89]:
#Définition du widget button
button = widgets.Button(description='Démarrer')

#Définition du widget affichage d'une sortie
cambridge = widgets.Output()
longman = widgets.Output()
linguee = widgets.Output()
tableau = widgets.Output()
    
def removeRegex(inputString):
    outputstring = ""
    for c in inputString:
        if c.isalpha():
            outputstring += c
    return outputstring

#Spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):
    
    path = "./../data/audio/"

    #Récupérer le texte saisi par l'utilisateur
    if len(textTarget.value) != 0:
        colToUse = 'hwd'
        targetWord = removeRegex(textTarget.value)
        
    else:
        colToUse = 'ipa'
    
    with cambridge:
        clear_output()
        urlCambridge = "https://dictionary.cambridge.org/fr/dictionnaire/anglais/"
        urlTarget = urlCambridge+targetWord
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'Cambridge : '+targetWord+'</a>'))

    with longman:
        clear_output()        
        urlLongman = "https://www.ldoceonline.com/dictionary/"
        urlTarget = urlLongman+targetWord
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'longman : '+targetWord+'</a>'))

        
    #Filtrage tableau
    buchananFiltered = buchanan[buchanan[colToUse].str.contains(textTarget.value, na=False, regex=True)]
    buchananFiltered = buchananFiltered[buchananFiltered['nbSyll'].between(wid_nbsyll.value[0], wid_nbsyll.value[1], inclusive=True)]

    with tableau:
        clear_output()
        display(buchananFiltered[['motVedette', 'nbSyll', 'pron', 'pronSampa', 'ipa']].head(n=wid_nbResult.value))
           
    for wordInfo in buchananFiltered.head(n=wid_nbResult.value).itertuples():
        getAudioInfo(wordInfo.motVedette).head()
        
        if getAudioInfo(wordInfo.motVedette).empty:
            print("pas d'informations pour", wordInfo.motVedette)
        
        else:
            row = [wordInfo.motVedette,
                   getAudioInfo(wordInfo.motVedette)[0:1].ipa.values[0],
                   getAudioInfo(wordInfo.motVedette)[0:1].pos.values[0],
                   "",
                   ""]
            
            items = [widgets.Label(row[0].upper()+":"), widgets.Label(row[2]), widgets.Label(" - "+row[1])]
            display(widgets.HBox(items))
            
            #AUDIO - get_url
            if len(getAudioInfo(wordInfo.motVedette)[0:1].audio_uk)>0:
                row[3] = getAudioInfo(wordInfo.motVedette)[0:1].audio_uk.values[0]
                audio_uk = True
                #DOWNLOAD MP3
                session_uk = HTMLSession()
                audio = session_uk.get(row[3])
                with open(path+wordInfo.motVedette+'_uk.mp3', 'wb') as f:
                    f.write(audio.content)
            if len(getAudioInfo(wordInfo.motVedette)[0:1].audio_us)>0:
                row[4] = getAudioInfo(wordInfo.motVedette)[0:1].audio_us.values[0]
                audio_us = True
                #DOWNLOAD MP3
                session_us = HTMLSession()
                audio = session_us.get(row[4])
                with open(path+wordInfo.motVedette+'_us.mp3', 'wb') as f:
                    f.write(audio.content)
            
            #AUDIO - player
            if audio_uk and audio_us:
                audio_widgets = []
                for i in range(2):
                    if i == 0:
                        #widget audio
                        outAudio = widgets.Output()
                        with outAudio:
                            display(Audio(url=row[3]))
                        
                        outSpec = widgets.Output()
    
                            
                        audio_widgets.append(widgets.VBox([widgets.Label("UK"), outAudio]))
                        
                    if i == 1:
                        #widget audio
                        outAudio = widgets.Output()
                        with outAudio:
                            display(Audio(url=row[4]))
                        
                        outSpec = widgets.Output()

                            
                        audio_widgets.append(widgets.VBox([widgets.Label("US"), outAudio]))
                    
                display(widgets.HBox(audio_widgets))
                
            elif audio_uk:
                #widget audio
                outAudio = widgets.Output()
                with outAudio:
                    display(Audio(url=row[3]))
                                        
                        
                outSpec = widgets.Output()

                display(widgets.VBox([widgets.Label("UK"), outAudio]))   
                
                
            else:
                #widget audio
                out = widgets.Output()
                with out:
                    display(Audio(url=row[4]))
                   
                        
                outSpec = widgets.Output()
                    
                display(widgets.VBox([widgets.Label("US"), outAudio])) 
                



button.on_click(on_button_clicked)


In [90]:
#Définition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='Orthographe',
       layout={'width': '300px'} )

ipaTarget = widgets.Text(
       value="",
       description='Prononciation',
       layout={'width': '300px'} )

#Sélection d'une plage de valeurs
rangeSyllMin = 1
rangeSyllMax = 27
optionsNbSyll = range(1, 27)
wid_nbsyll = widgets.SelectionRangeSlider(
    options=optionsNbSyll,
    index=(min(optionsNbSyll)-1,max(optionsNbSyll)-1),
    description='Nb syllabes',
    disabled=False
)


#Filtre pour limiter le data frame en sorti
nbColonne = widgets.IntSlider(
    max = 10, value = 1,
    description='Limit Data', 
    disabled=False
)


In [91]:
#Définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

#Définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()
button.on_click(on_button_clicked)

#Widgets de filtre input
vbFiltres = widgets.HBox([textTarget, ipaTarget])
vbFiltres2 = widgets.HBox([wid_nbsyll, wid_nbResult, button])
#Widgets d'output
vbOuts = widgets.VBox([cambridge, longman, linguee, tableau])

hbAllWidgets = widgets.VBox([vbFiltres,vbFiltres2, vbOuts])
hbAllWidgets

IndexError: list index out of range